In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sivarazadi/wikiart-art-movementsstyles")

print("Path to dataset files:", path)

100%|██████████| 27.0G/27.0G [06:06<00:00, 79.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sivarazadi/wikiart-art-movementsstyles/versions/2


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm


In [ ]:
IMG_SIZE = 128
BATCH_SIZE = 16

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])


data_dir = "/root/.cache/kagglehub/datasets/sivarazadi/wikiart-art-movementsstyles/versions/2"

train_dataset = ImageFolder(data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
num_classes = len(train_dataset.classes)

print("Classes:", train_dataset.classes)


Classes: ['Academic_Art', 'Art_Nouveau', 'Baroque', 'Expressionism', 'Japanese_Art', 'Neoclassicism', 'Primitivism', 'Realism', 'Renaissance', 'Rococo', 'Romanticism', 'Symbolism', 'Western_Medieval']


In [ ]:
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, num_classes)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 181MB/s]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)


In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss:.4f}")


  0%|          | 0/2657 [00:00<?, ?it/s]

Epoch 1/5, Loss: 3921.0714


  0%|          | 0/2657 [00:00<?, ?it/s]

Epoch 2/5, Loss: 3018.6687


  0%|          | 0/2657 [00:00<?, ?it/s]

Epoch 3/5, Loss: 2417.6616


  0%|          | 0/2657 [00:00<?, ?it/s]

Epoch 4/5, Loss: 1795.8637


  0%|          | 0/2657 [00:00<?, ?it/s]

Epoch 5/5, Loss: 1275.9885


In [ ]:
torch.save(model.state_dict(), "art_style_resnet18_1.pth")
print("Model saved!")


Model saved!


In [ ]:
# del train_loader, train_dataset
# import gc, torch
# gc.collect()
# torch.cuda.empty_cache()


In [ ]:
import torch
import numpy as np
from PIL import Image

def predict_image(path):
    img = Image.open(path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(img)
        probs = torch.softmax(output, dim=1)[0]
        pred_idx = probs.argmax().item()
        pred_conf = probs[pred_idx].item()

    return train_dataset.classes[pred_idx], pred_conf


In [ ]:
example_path = train_dataset.imgs[0][0]  # path only
style, conf = predict_image(example_path)

print("Example prediction:")
print("Style:", style)
print("Confidence:", conf)


Example prediction:
Style: Academic_Art
Confidence: 0.9966248273849487


In [19]:
import gradio as gr
import numpy as np
from PIL import Image
import torch

def classify_image(img):
    # Convert PIL → tensor
    img = img.convert("RGB")

    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor()
    ])

    img_tensor = transform(img).unsqueeze(0).to(device)

    # Prediction
    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        probs = torch.softmax(output, dim=1)[0]


    conf, idx = torch.max(probs, dim=0)
    class_name = train_dataset.classes[idx]

    return {class_name: float(conf)}


demo = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(),
    title="Art Style Classifier"
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1446af195dffd2381.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
